In [1]:
import os, sys, pickle, random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.cm as cm
from tqdm import tqdm
from pathlib import Path
import mne
if os.getcwd().split("/")[-1] != 'BENDR-XAI': os.chdir("../")
from notebooks.utils import *

In [10]:
# Read files on each each line in tuh_final_selected.txt
selected = []
with open('data_process/tuh_final_selected.txt' , 'r') as f:
    selected.append(f.read().split('/')[-1].strip())

In [14]:
# Delete files in data/tuh_eeg_clean that are not in selected

for file in os.listdir('data/tuh_eeg_clean/'):
    if file not in selected:
        os.remove('data/tuh_eeg_clean/' + file)

In [2]:
edf_dir = Path("/scratch/s194260/tuh_eeg")
edf_files = [edf_dir / file for file in os.listdir(edf_dir)]

In [3]:
subjects_dir, subject, trans, src_path, bem_path = get_fsaverage()
info = read_TUH_edf(edf_files[0]).info

In [13]:
for file in tqdm(edf_files):
    info = read_TUH_edf(file).info
    if not all([np.all(~np.isnan(ch['loc'][:3])) for ch in info['chs']]):
        print(file)
        break

100%|██████████| 200/200 [01:30<00:00,  2.22it/s]


In [10]:
# We use T3 and T4 instead of T7 and T8
channels_order = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']

data = np.load("data/HCPMMP1_combined_8.0_12.0_220503_280423.npy", allow_pickle=True).item()

minimum = -0.00125
maximum = 0.00125

for edf_file, result_dict in tqdm(data.items(), total=len(data)):
    raw = read_TUH_edf(f"/scratch/s194260/tuh_eeg/{edf_file}", low_pass=None)
    try:
        raw = raw.reorder_channels(channels_order)
    except ValueError:
        continue
    
    for anno in result_dict['annotations']['T0']:
        window = get_window(raw, anno).get_data()
        window = (window - minimum) / (maximum - minimum) * 2 - 1

    #for anno in tqdm(result_dict['annotations']['T0']):
    #    window = get_window(raw, anno).get_data()
    #    minimum = np.minimum(window.min(axis=1), minimum)
    #    maximum = np.maximum(window.max(axis=1), maximum)
        
#minimum = minimum.reshape(-1, 1)
#maximum = maximum.reshape(-1, 1)

(19, 1024)

In [5]:
# Read the files on each line in /home/s194260/BENDR-XAI/tuh_selected5.txt
files = []
with open('tuh_selected5.txt', 'r') as f:
    for line in f:
        files.append(line.strip())

In [6]:
import os
from tqdm import tqdm

In [7]:
# Copy each file in files to /scratch/s194260/tuh_eeg
for file in tqdm(files):
    os.system(f"cp {file} /scratch/s194260/tuh_eeg")

  2%|▏         | 167/8518 [00:35<23:50,  5.84it/s]  

In [2]:
# Make a list of path to all edf files in the subfolders of /nobackup/tsal-tmp/tuh_eeg
# files = []

# with tqdm() as pbar:
#     for root, dirs, file in os.walk("/nobackup/tsal-tmp/tuh_eeg"):
#         for name in file:
#             if name.endswith(".edf"):
#                 files.append(os.path.join(root, name))
#                 pbar.update(1)

# Save list of files as pickle
# import pickle
# with open('tuh_files.pkl', 'wb') as f:
#     pickle.dump(files, f)
    
# Load list of files from pickle
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)
    
# random.shuffle(files)

In [3]:
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)

# Copy all files in tuh_final_selected.txt to data/tuh_eeg/
# with open('tuh_final_selected.txt', 'r') as f:
#     selected = f.read().splitlines()

# selected = [file.split("/")[-1] for file in selected]

# Delete all files in /scratch/s194260/tuh_eeg/ that are not in selected
# for file in os.listdir('/scratch/s194260/tuh_eeg/'):
#     if file not in selected:
#         os.remove('/scratch/s194260/tuh_eeg/' + file)